In [1]:
import psycopg2
import pickle
import 

/home/lmoraes/miniconda/envs/machine_teaching/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [52]:
import pickle

with open("clustering_bcc.pkl", "rb") as pklfile:
    df = pickle.load(pklfile)

In [53]:
df

,X,coherence_k,coherence_med,coherence_samples,coherence_std,coherence_time,dataset,distance,gap,gap_std,...,is_binary,k,method,min_df,model,silhouette,silhouette_samples,silhouette_time,vectorizer,y
0,"[[0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,...",2,-1.025149,"[-1.33876579541, -0.711531716001]",0.313617,0.069484,bbc_5,cosine,3,2,...,True,2,nmf,0.25,b'gANjc2tsZWFybi5kZWNvbXBvc2l0aW9uLm5tZgpOTUYK...,0.022251,"[-0.0244050748579, -0.175526370274, -0.1796602...",0.186973,CountVectorizer,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[[0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,...",3,-0.866658,"[-1.23776593072, -0.837921832252, -0.866657976...",0.182093,0.063246,bbc_5,cosine,3,2,...,True,3,nmf,0.25,b'gANjc2tsZWFybi5kZWNvbXBvc2l0aW9uLm5tZgpOTUYK...,0.004032,"[-0.0151932077764, -0.184611501235, -0.1045330...",0.259234,CountVectorizer,"[0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, ..."
2,"[[0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,...",4,-0.989920,"[-1.41252032558, -0.9775669303, -0.83623701356...",0.214730,0.063187,bbc_5,cosine,3,2,...,True,4,nmf,0.25,b'gANjc2tsZWFybi5kZWNvbXBvc2l0aW9uLm5tZgpOTUYK...,-0.011443,"[-0.0260368780685, -0.145409563405, -0.0793537...",0.226391,CountVectorizer,"[0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0, 2, 0, ..."
3,"[[0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,...",5,-0.905160,"[-1.47605977903, -0.770253920926, -0.841561976...",0.263860,0.050849,bbc_5,cosine,3,2,...,True,5,nmf,0.25,b'gANjc2tsZWFybi5kZWNvbXBvc2l0aW9uLm5tZgpOTUYK...,-0.005971,"[-0.0311539014196, 0.122621057355, -0.06601606...",0.218311,CountVectorizer,"[0, 1, 2, 0, 4, 2, 0, 4, 4, 0, 2, 2, 4, 2, 0, ..."
4,"[[0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,...",6,-0.745208,"[-1.34229746791, -0.711221214307, -0.740745711...",0.237375,0.050403,bbc_5,cosine,3,2,...,True,6,nmf,0.25,b'gANjc2tsZWFybi5kZWNvbXBvc2l0aW9uLm5tZgpOTUYK...,-0.009806,"[-0.0433505323497, 0.108325138605, -0.09414101...",0.228979,CountVectorizer,"[0, 1, 2, 4, 4, 4, 0, 4, 4, 4, 5, 0, 4, 2, 0, ..."
5,"[[0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,...",7,-0.761277,"[-1.26826816313, -0.629970076663, -0.720572928...",0.519948,0.047421,bbc_5,cosine,3,2,...,True,7,nmf,0.25,b'gANjc2tsZWFybi5kZWNvbXBvc2l0aW9uLm5tZgpOTUYK...,-0.024487,"[0.0676231557333, -0.0413591812028, -0.0791788...",0.227516,CountVectorizer,"[5, 5, 2, 0, 5, 5, 5, 6, 5, 0, 5, 2, 0, 5, 0, ..."
6,"[[0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,...",8,-0.724926,"[-1.09180670549, -0.36710323315, -0.5694325181...",0.392653,0.046617,bbc_5,cosine,3,2,...,True,8,nmf,0.25,b'gANjc2tsZWFybi5kZWNvbXBvc2l0aW9uLm5tZgpOTUYK...,-0.024007,"[-0.0406978010112, -0.220711312836, -0.1271383...",0.222543,CountVectorizer,"[0, 0, 2, 4, 4, 4, 0, 6, 4, 4, 4, 0, 4, 6, 0, ..."
7,"[[0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,...",9,-0.874156,"[-1.07761387451, -0.542930774018, -0.475446168...",0.945959,0.049305,bbc_5,cosine,3,2,...,True,9,nmf,0.25,b'gANjc2tsZWFybi5kZWNvbXBvc2l0aW9uLm5tZgpOTUYK...,-0.032965,"[-0.024605510271, -0.200221722097, -0.13764394...",0.220606,CountVectorizer,"[0, 0, 2, 6, 0, 6, 0, 6, 0, 2, 0, 0, 6, 6, 0, ..."


In [6]:
connection = psycopg2.connect(user = "machineteaching",
                                  password = "***REMOVED***",
                                  host = "localhost",
#                                   port = "5432",
                                  database = "machineteaching")
connection.autocommit=True
cursor = connection.cursor()

In [56]:
create_table_query = '''CREATE TABLE EXPERIMENTS
    (EXPERIMENT_ID SERIAL PRIMARY KEY,
    DATASET TEXT NOT NULL,
    X REAL ARRAY NOT NULL,
    Y REAL ARRAY NOT NULL,
    VECTORIZER TEXT NOT NULL,
    IS_BINARY BOOLEAN NOT NULL,
    MIN_DF REAL NOT NULL,
    DISTANCE TEXT NOT NULL,
    K INTEGER NOT NULL,
    METHOD TEXT NOT NULL,
    MODEL  bytea NOT NULL,
    GAP REAL NOT NULL,
    GAP_STD REAL NOT NULL,
    GAP_TIME REAL NOT NULL,
    GAP_K INTEGER NOT NULL,
    SILHOUETTE REAL NOT NULL,
    SILHOUETTE_SAMPLES REAL ARRAY NOT NULL,
    SILHOUETTE_TIME REAL NOT NULL,
    COHERENCE_SAMPLES REAL ARRAY NOT NULL,
    COHERENCE_MED REAL NOT NULL,
    COHERENCE_STD REAL NOT NULL,
    COHERENCE_TIME REAL NOT NULL,
    COHERENCE_K INTEGER NOT NULL)
    '''

# create_table_query = '''CREATE TABLE EXPERIMENTS_TEMP
#     (EXPERIMENT_ID SERIAL PRIMARY KEY,
#     X REAL ARRAY NOT NULL,
#     Y REAL ARRAY NOT NULL,
#     VECTORIZER TEXT NOT NULL,
#     IS_BINARY BOOLEAN NOT NULL)
#     '''

cursor.execute(create_table_query)
connection.commit()

b"INSERT INTO EXPERIMENTS (X, coherence_k, coherence_med, coherence_samples, coherence_std, coherence_time, dataset, distance, gap, gap_std, gap_time, is_binary, k, method, min_df, model, silhouette, silhouette_samples, silhouette_time, vectorizer, y) VALUES (ARRAY[ARRAY[0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,1,1,0,0,1,1,0,0,0,0,1,0,1,1,0,1,0,1,0,0,0,1,0,1,1,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,1,0,1,1,1,0,0,0,0],ARRAY[0,0,0,0,0,1,1,1,0,1,1,0,1,0,0,0,0,1,0,1,0,1,0,1,1,1,1,1,1,1,0,0,1,0,1,1,1,1,0,1,0,0,1,1,1,1,0,1,1,1,1,1,1,0,0,0,1,0,1,0,0,0,1,1,1,0,0,1,1,0,0,1,1,0,1,0,1,0,1,0,0,0,1,1,0,1,0,1,0,0,0],ARRAY[0,0,1,1,0,0,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,0,1,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,0,1,0,0,1,0,1,1,0,0,0,0,1,1,1,0,0,1,1,1,0,1,0,0,1,1,1,1,0,0,0,0,1,0,1,1,1,0,1,0,0],ARRAY[0,0,0,1,1,0,1,1,1,1,1,1,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,1,0,0,0,0,1,0,0,1,0,1,1,0,1,1,1,0,0,0,1,1,0,1,1,0,0,1,1,1,1,0,1,0

In [46]:
cursor.execute("select model from experiments")
d = cursor.fetchall()

In [47]:
import base64

In [ ]:
m = pickle.loads(base64.b64decode(d[0][0]))

In [51]:
m.fit()

TypeError: fit() missing 1 required positional argument: 'X'

In [10]:
import numpy as np

In [42]:
insert_query = "INSERT INTO EXPERIMENTS "
column_value = []
insert_format = []
query_values = []
extra = []

for row in df.iterrows():
    query = []
#     for col in df.columns:
    for col in df.columns:
        if isinstance(row[1][col], np.ndarray):
            query_values.append(row[1][col].tolist())
        else:
            query_values.append(row[1][col])
        column_value.append(col)
        insert_format.append("%s")
    
insert_query += "(" + ", ".join(column_value) + ") VALUES "
insert_query += "(" + ", ".join(insert_format) + ")"
query_values = tuple(query_values)

In [43]:
insert_query

'INSERT INTO EXPERIMENTS (X, coherence_k, coherence_med, coherence_samples, coherence_std, coherence_time, dataset, distance, gap, gap_std, gap_time, is_binary, k, method, min_df, model, silhouette, silhouette_samples, silhouette_time, vectorizer, y) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [13]:
query_values[1]

2

In [30]:
cursor.mogrify('INSERT INTO EXPERIMENTS_TEMP (X) VALUES (%s)', (d,))

ProgrammingError: can't adapt type 'numpy.ndarray'